In [ ]:
# %matplotlib qt4
# doesn't work with Tkinter

In [1]:
# load modules

import Tkinter as tk
import tkFileDialog
# import pyqtgraph as pg # doesn't work with the necessary library for matplotlib
import os
# import cv2 as cv
import numpy as np

In [2]:
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib tk

/Users/callinswitzer/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
%qtconsole

In [5]:
# %qtconsole
sns.set_style('white')

# scikit image
import skimage
from skimage import io

In [6]:
def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [81]:
aa = file_choose()
print aa

/Users/callinswitzer/Dropbox/KalmiaProject/KalmiaFruits_28Sept2016/39_60_A__2.jpg


In [82]:
# import image
camera = io.imread(aa)


In [83]:
io.imshow(camera[0])

In [84]:
img_gray = skimage.color.rgb2gray(camera[0])

In [85]:
io.imshow(img_gray)

In [86]:
io.imshow(camera[0][:,:,2])

In [89]:
from skimage import filters
val = filters.threshold_otsu(camera[0]) # calculate threshold with Otsu's method
mask = camera[0] < val
io.imshow(mask)

In [95]:
from skimage import segmentation
mask = img_gray < filters.threshold_otsu(img_gray) # get segmented image
clean_border = segmentation.clear_border(mask) # clear border
io.imshow(mask)

In [100]:
from skimage import morphology
from skimage import segmentation
clean_border = segmentation.clear_border(mask) # clear border

In [130]:
# these steps get rid of the little leaflets protruding from the circles
# unfortunately this takes a long time
erIm = morphology.erosion(mask, morphology.disk(2))
#erIm = morphology.erosion(erIm, morphology.disk(5))
diIm = morphology.dilation(erIm, morphology.disk(2))
#diIm = morphology.dilation(diIm, morphology.disk(5))
io.imshow(diIm)
io.show()

In [112]:
# this binary erode could be better
b_open = morphology.binary_opening(mask, selem = morphology.diamond(15))
io.imshow(b_open)

In [127]:
io.imshow(diIm)
io.show()

In [98]:
# label blobs in image
plt.clf()
lab_img = morphology.label(diIm, connectivity=2)
imgplt = io.imshow(lab_img)
imgplt.set_cmap('gist_ncar')

In [54]:
# regionprops
props = skimage.measure.regionprops(lab_img)

In [102]:
#plt.clf()
plt.plot(props[26].centroid[0],props[26].centroid[1] , 'bo', ms = 50)
plt.show()

In [73]:
# canny filter to find edges
from skimage import feature


# Generate noisy image of a square
im = img_gray

# Compute the Canny filter for two values of sigma
edges1 = feature.canny(im)
edges2 = feature.canny(im, sigma=3)

# display results (open new figure)
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(8, 3),
                                    sharex=True, sharey=True)

ax1.imshow(im, cmap=plt.cm.gray)
ax1.axis('off')
ax1.set_title('noisy image', fontsize=20)

ax2.imshow(edges1, cmap=plt.cm.gray)
ax2.axis('off')
ax2.set_title('Canny filter, $\sigma=1$', fontsize=20)

ax3.imshow(edges2, cmap=plt.cm.gray)
ax3.axis('off')
ax3.set_title('Canny filter, $\sigma=3$', fontsize=20)

fig.tight_layout()

plt.show()

In [80]:
# fill holes after canny filter
from scipy import ndimage as ndi
fill_can = ndi.binary_fill_holes(edges2)
plt.imshow(fill_can) # doesn't work, b/c some loops aren't completely closed -- should use stobel instead

#io.imshow(morphology.label(fill_can, connectivity = 2) )